# **RECORDLINKAGE  TOOL**

### Required imports

In [ ]:
import pandas as pd
import numpy as np
import recordlinkage
import interpret
from sklearn.model_selection import train_test_split

from recordlinkage import datasets
from raiwidgets import ErrorAnalysisDashboard
from interpret.ext.blackbox import MimicExplainer
from interpret_community.common.constants import ModelTask


### Loading datasets 

In [ ]:
first_febrl_dataset, second_febrl_dataset = datasets.load_febrl4(return_links=False)

In [ ]:
indexes_frst_febrl_dataset = sorted(first_febrl_dataset)
indexes_scd_febrl_dataset = sorted(second_febrl_dataset.index)
true_links = list(zip(indexes_frst_febrl_dataset, indexes_scd_febrl_dataset))

In [ ]:
first_febrl_dataset.head()

In [ ]:
second_febrl_dataset.head()

**PREPROCESSING**

**INDEXING STEP**

L'indexation n'est faisable que sur un ou deux jeu de données. Il propose 4 méthodes d'indexations:
- Full : renvoie toutes les combinaisons de paires possibles
- Block : renvoie tous les éléments qui concordent par rapport aux variables données en entrée 
- SortedNeighbourhood : renvoie tous les éléments qui concordent par rapport aux variables données en entrée et celles dans leur voisinnage
- Random : renvoi des paires crées aléatoirement

Ici, nous testons la méthode du block en utilisant les colonnes 'given_name' et 'address_1'. Avec le blocage sur le nom et adresse, les candidats seront filtrés pour n'inclure que ceux dont les valeurs sont similaires.

In [ ]:
indexer = recordlinkage.Index()
indexer.block(on='given_name')
indexer.block(on='address_1')
candidate_pairs = indexer.index(first_febrl_dataset, second_febrl_dataset)

Dans le cas ou les données sont susceptible de contenir des erreurs de typographies, utiliser la méthode SortedNeighborhood peut ajouter une certaine flexibilité pour les fautes d'orthographe mineures. 

**COMPARAISON**

La méthode suivante consiste à comparer les paires en utilisant Compare. </br>
Nous pouvons définir plusieurs options pour la façon dont nous voulons comparer les colonnes de données.

In [ ]:
comp = recordlinkage.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler')
comp.string('surname', 'surname', method='jarowinkler')
comp.string('address_1', 'address_1', method='levenshtein')
comp.exact('soc_sec_id', 'soc_sec_id')

# the method .compute() returns the DataFrame with the feature vectors.
features = comp.compute(candidate_pairs, first_febrl_dataset, second_febrl_dataset)

**CLASSIFICATION**

Les vecteurs de scores de similarités sont donnés en entrée au classifieur (supervisé ou non supervisé). 

In [ ]:
#kmeans_clf = recordlinkage.KMeansClassifier()
#links_pred_kmeans = kmeans_clf.fit_predict(features).values

In [ ]:
# Split data into train and test
x_train, x_test, y_train, y_test = train_test_split(candidate_pairs, true_links, test_size=0.5, random_state=0)

In [ ]:
clf = recordlinkage.SVMClassifier()
model = clf.fit(x_train, y_train)

In [ ]:
# Train the LightGBM surrogate model using MimicExplaner
model_task = ModelTask.Classification
explainer = MimicExplainer(model, x_train, LGBMExplainableModel,
                           augment_data=True, max_num_of_augmentations=10,
                           features=feature_names, classes=classes, model_task=model_task)

In [ ]:
global_explanation = explainer.explain_global(x_test)
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

**VISUALIZE**

In [ ]:
ErrorAnalysisDashboard(global_explanation, model, dataset=x_test, true_y=y_test)